# Часть I. Односторонние разностные схемы.

Напишите функцию `deriv`, которая вычисляет производную аргумента в заданной точке $x$, используя односторонню разностную схему с заданным шагом $h$ и степенью аппроксимации $O(h^2)$.

In [1]:
def deriv(f, x, h):
    """ Вычисляет производную `f` в точке `x` с шагом `h`.
    Вычисляет производную, используя односторонню разностную схему со степенью аппроксимации $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        Функция, которую нужно продифференцировать
    x : float
        Точка, в которой нужно дифференцировать функцию
    h : float
        Шаг
        
    Rerurns
    -------
    fder : производная f(x) в точке x с шагом h.
    """
    return (-0.5 * f(x + 2 * h) + 2 * f(x + h) - 1.5 * f(x)) / h

#### Тест I.1

Проверьте ваш алгоритм на простом примере: продифференцируйте $f(x) = x^3$ в точке $x=0$. Прокомментируйте, совпадает ли результат с ожидаемым $f'(x) = 0$ при стремлении $h\to 0$.

 (10% итоговой оценки)

In [2]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.0002
0.001000 --  -2e-06
0.000100 --  -2e-08
0.000010 --  -2e-10


Совпадает с точностью до $h^2$, как мы и предполагали

### Тест I.2

Теперь попробуйте немного более сложную функцию $f(x) = x^2 \log{x}$. Оцените значение производной в точке $x=1$, используя двухточечную и трехточечную схемы. Для обеих схем оцените значение $h$, при котором ошибка перестанет падать.

(15% итоговой оценки)

In [3]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

In [148]:
import matplotlib.pyplot as plt
import numpy as np

def deriv_2(f, x, h):
    return (f(x+h) - f(x)) / h

x = 1
h = 0.001
H = 0.001

delta1 = 1000
delta2 = 100
Delta1 = 1000
Delta2 = 100

while delta1 > delta2:
    delta1 = delta2
    h = 0.1 * h
    delta2 = abs(deriv(f, x, h) - fder(x))

while Delta1 > Delta2:
    Delta1 = Delta2
    H = 0.1 * H
    Delta2 = abs(deriv_2(f, x, h) - fder(x))


print("Три точки:", h)
print("Две точки:", H)


Три точки: 1.0000000000000002e-06
Две точки: 1e-05


### Тест I.3 

Теперь продифференцируйте $x^2 \log(x)$ в $x=0$. Используйте трехточечную схему. Заметьте, что В нуле функцию нужно доопределить явным образом. Проверьте шкалированные ошибки. Объясните полученные результаты.

(25% итоговой оценки)

In [142]:
def f(x):
    if x == 0:
        # предел $x^2 log(x)$ при $x-> 0$ равен нулю, хотя log(x) не определен в x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.01386
0.001000 -- -0.001386
0.000100 -- -0.0001386
0.000010 -- -1.386e-05


... ENTER YOUR EXPLANATION HERE ...

# Part II. Midpoint rule 

Напишите функцию, вычисляющую определенный интеграл, используя "midpoint rule", с точностью до заданной погрешности $\epsilon$. Оцените ошибку, сравнивая значения интеграла для $N$ и $2N$ элементарных интервалов.

In [69]:
def midpoint_rule(func, a, b, eps):
    """ Вычисляет интеграл f от a до b используя "midpoint rule".
    
    Параметры
    ----------
    func : callable
        Функция, которую нужно проинтегрировать
    a : float
        Нижний предел интегрирования
    b : float
        Верхний предел интегрирования
    eps : float
        Ожидаемая ошибка оценки
        
    Возвращает
    -------
    integral : float
        Оценка интеграла $\int_a^b f(x) dx$.
    """
    #n - число столбиков, добавляем и вычитаем, чтобы были x только в нужных точках
    answer = np.inf
    integral = (b - a) * func((b - a) / 2)
    n = 1
    while abs(integral - answer) > eps:
        answer = integral
        n = n * 2
        d = (b - a) / n
        x = np.linspace(a + d * 0.5, b - d * 0.5, n)
        
        integral = sum([func(i) * d for i in x])
    return integral, n

In [96]:
print(midpoint_rule(lambda x: x**5, -5, 5, 1e-3)[0])
print(midpoint_rule(lambda x: np.cos(x), -np.pi, np.pi, 1e-3)[0])
print(midpoint_rule(lambda x: x, 0, 4, 1e-3)[0])
#все ответы совпадают
#выведем ответ такой формулы: значение интеграла * число шагов в квадрате; сравним ответы для разных eps
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-3)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-3)[1]**2)
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-4)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-4)[1]**2)
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-5)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-5)[1]**2)

0.0
0.0
8.0
-2.25
-2.25
-2.25


### Тест II.1

Протестирутйте ваш алгоритм на простом интеграле, который вы можете посчитать аналитически.

Сравните скорость сходимости с ожидаемой $O(N^{-2})$ в зависимости от количества интервалов, необходимых для заданной точности $\epsilon$.

Сравните полученный результат с ответом, вычисленным руками. Попадает ли результат в интервал заданной ошибки?

(20% итоговой оценки)


In [146]:
print(midpoint_rule(lambda x: x**5, -5, 5, 1e-3)[0])
print(midpoint_rule(lambda x: np.cos(x), -np.pi, np.pi, 1e-3)[0])
print(midpoint_rule(lambda x: x, 0, 4, 1e-3)[0])
#все ответы совпадают
#выведем ответ такой формулы: значение интеграла * число шагов в квадрате; сравним ответы для разных eps
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-3)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-3)[1]**2)
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-4)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-4)[1]**2)
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-5)[0] - 3**3 / 3) * midpoint_rule(lambda x: x**2, 0, 3, 1e-5)[1]**2)
# теперь посмотрим на интервал ошибки
print((midpoint_rule(lambda x: x**2, 0, 3, 1e-5)[0] - 3**3 / 3)) # ошибка порядка -6 степени, значит попадает в интервал

0.0
0.0
8.0
-2.25
-2.25
-2.25
-2.1457672119140625e-06


... ENTER YOUR CODE AND COMMENTS HERE ...

### Тест II.2

Используя ваш алгоритм, посчитайте значение

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

с точностью до $\epsilon=10^{-4}$.

Заметим, что интеграл содержит интегрируемую особенность в нижнем пределе. Выполните вычисление двумя способами: во первых, посчитайте интеграл "в лоб", во вторых, вычтите особенность из подынтегрального выражения. Сравните количество необходимых итераций для достижения заданной точности $\epsilon$.

(30% итоговой оценки)

In [104]:
I, n = midpoint_rule(lambda x: np.sin(x**0.5) / x, 0, 1, 1e-4)

In [138]:
I_s, n_s = midpoint_rule(lambda x: (np.sin(x**0.5) - x**0.5)/ x, 0, 1, 1e-4) # находим интеграл - сингулярность
I_s = I_s + 2  # интеграл сингулярности = 2

In [139]:
print(n, I)     # ответ и число точек влоб
print(n_s, I_s) # ответ и число интервалов вычисляя отдельно сингулярность 
print(n / n_s)  # вычтя сингулярность количество интервалов уменьшилось в 262144 раза, это 18 РАЗ!!!

8388608 1.891957289204461
32 1.892113005639548
262144.0
